<a href="https://colab.research.google.com/github/Abhiroobini/Speech-Speech-Translation-french-to-english-/blob/main/Final_t%26s_project(french_to_english).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q openai-whisper
!pip install -q openai-whisper transformers sentencepiece gtts torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 10.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.4 MB/s 

In [4]:
pip install streamlit whisper transformers gtts


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.8 MB/s eta 0:00:00
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=572ddf199d4e65a0a29f3a6b293a1184ac51e5d432807846b8ffb02c56ac0022
  Stored in directory: /root/.cache/pip/wheels/21/65/ee/4e6672aabfa486d3341a39a04f8f87c77e5156149299b5a7d0
Successfully built whisper


In [5]:
import os
import whisper
import tempfile
import streamlit as st
from transformers import MarianMTModel, MarianTokenizer
from gtts import gTTS

@st.cache_resource
def load_whisper_model():
    return whisper.load_model("base")

@st.cache_resource
def load_translation_model(source_lang, target_lang):
    model_name = f"Helsinki-NLP/opus-mt-{source_lang}-{target_lang}"
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    return tokenizer, model

def transcribe_audio(audio_path):
    model = load_whisper_model()
    result = model.transcribe(audio_path)
    return result["text"], result["language"]

def translate_text(text, source_lang, target_lang):
    tokenizer, model = load_translation_model(source_lang, target_lang)
    tokens = tokenizer(text, return_tensors="pt", padding=True)
    translated = model.generate(**tokens)
    return tokenizer.decode(translated[0], skip_special_tokens=True)

def text_to_speech(text, lang_code):
    tts = gTTS(text=text, lang=lang_code)
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as fp:
        tts.save(fp.name)
        return fp.name

st.set_page_config(page_title="Speech-to-Speech Translator", layout="centered")
st.title("🎤 French to English Speech Translator")

uploaded_files = st.file_uploader("Upload a French audio file ", type=["mp3"], accept_multiple_files=True)

if uploaded_files:
    if len(uploaded_files) > 20:
        st.warning("Please upload a maximum of 20 files.")
    else:
        file_names = [file.name for file in uploaded_files]
        selected_filename = st.selectbox("Select one audio file to translate:", file_names)

        selected_file = next(file for file in uploaded_files if file.name == selected_filename)

        st.markdown("---")
        st.subheader(f"📁 Selected File: `{selected_file.name}`")

        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as tmp_audio:
            tmp_audio.write(selected_file.read())
            audio_path = tmp_audio.name

        try:
            with st.spinner("Transcribing audio..."):
                original_text, detected_lang = transcribe_audio(audio_path)
                st.success(f"Detected Language: `{detected_lang}`")
                st.text_area("📝 Transcribed Text", original_text, height=150)

            with st.spinner("Translating text..."):
                translated_text = translate_text(original_text, detected_lang, "en")
                st.text_area("🌍 Translated Text", translated_text, height=150)

            with st.spinner("Generating audio..."):
                translated_audio_path = text_to_speech(translated_text, "en")
                audio_file = open(translated_audio_path, 'rb')
                audio_bytes = audio_file.read()
                st.audio(audio_bytes, format='audio/mp3')
                audio_file.close()

        except Exception as e:
            st.error(f"❌ Error processing `{selected_file.name}`: {e}")



2025-05-15 12:22:50.694 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 12:22:50.700 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 12:22:51.135 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-05-15 12:22:51.140 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 12:22:51.141 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 12:22:51.142 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 12:22:51.143 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

In [6]:
!pip install streamlit whisper transformers gtts
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙
added 22 packages in 2s
⠙
⠙3 packages are looking for funding
⠙  run `npm fund` for details
⠙

In [7]:
!pip install -q streamlit openai-whisper transformers sentencepiece gtts pyngrok
!apt-get install -y ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [8]:
code = '''
import os
import whisper
import tempfile
import streamlit as st
from transformers import MarianMTModel, MarianTokenizer
from gtts import gTTS

# --- Load Models ---
@st.cache_resource
def load_whisper_model():
    return whisper.load_model("base")

@st.cache_resource
def load_translation_model(source_lang, target_lang):
    model_name = f"Helsinki-NLP/opus-mt-{source_lang}-{target_lang}"
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    return tokenizer, model

# --- Helper Functions ---
def transcribe_audio(audio_path):
    model = load_whisper_model()
    result = model.transcribe(audio_path)
    return result["text"], result["language"]

def translate_text(text, source_lang, target_lang):
    tokenizer, model = load_translation_model(source_lang, target_lang)
    tokens = tokenizer(text, return_tensors="pt", padding=True)
    translated = model.generate(**tokens)
    return tokenizer.decode(translated[0], skip_special_tokens=True)

def text_to_speech(text, lang_code):
    tts = gTTS(text=text, lang=lang_code)
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as fp:
        tts.save(fp.name)
        return fp.name

# --- Streamlit UI ---
st.set_page_config(page_title="Speech-to-Speech Translator", layout="centered")
st.title("🎤 French to English Speech Translator")

uploaded_files = st.file_uploader("Upload a French audio file ", type=["mp3"], accept_multiple_files=True)

# After uploading files
if uploaded_files:
    if len(uploaded_files) > 20:
        st.warning("Please upload a maximum of 20 files.")
    else:
        file_names = [file.name for file in uploaded_files]
        selected_filename = st.selectbox("Select one audio file to translate:", file_names)

        selected_file = next(file for file in uploaded_files if file.name == selected_filename)

        st.markdown("---")
        st.subheader(f"📁 Selected File: `{selected_file.name}`")

        # Save to temp
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as tmp_audio:
            tmp_audio.write(selected_file.read())
            audio_path = tmp_audio.name

        try:
            with st.spinner("Transcribing audio..."):
                original_text, detected_lang = transcribe_audio(audio_path)
                st.success(f"Detected Language: `{detected_lang}`")
                st.text_area("📝 Transcribed Text", original_text, height=150)

            with st.spinner("Translating text..."):
                translated_text = translate_text(original_text, detected_lang, "en")
                st.text_area("🌍 Translated Text", translated_text, height=150)

            with st.spinner("Generating audio..."):
                translated_audio_path = text_to_speech(translated_text, "en")
                audio_file = open(translated_audio_path, 'rb')
                audio_bytes = audio_file.read()
                st.audio(audio_bytes, format='audio/mp3')
                audio_file.close()

        except Exception as e:
            st.error(f"❌ Error processing `{selected_file.name}`: {e}")


'''

with open("app.py", "w") as f:
    f.write(code)

print("✅ app.py saved successfully!")


✅ app.py saved successfully!


In [9]:
from pyngrok import ngrok

ngrok.set_auth_token("2x39wFGd5GCHA0GJhMOqbsUIO1R_52ScG3gTgokaisLTW2aaG")


In [10]:
for tunnel in ngrok.get_tunnels():
    ngrok.disconnect(tunnel.public_url)


In [11]:
from pyngrok import ngrok
import subprocess
import threading
import time

def run():
    subprocess.run(["streamlit", "run", "app.py", "--server.port", "8501"])

threading.Thread(target=run).start()

time.sleep(5)

public_url = ngrok.connect(8501)
print("🌐 Public URL:", public_url)


🌐 Public URL: NgrokTunnel: "https://dd86-34-82-48-101.ngrok-free.app" -> "http://localhost:8501"


In [13]:
!pip install jiwer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.4 MB/s eta 0:00:00


In [15]:
!pip install pydub
!apt-get install ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [17]:
import os
import pandas as pd
from gtts import gTTS
import time  # Import time module

# Setup: Define directory and phrases
dataset_dir = "french_audio_dataset"
os.makedirs(dataset_dir, exist_ok=True)

phrases = [
    ("Bonjour tout le monde", "Hello everyone"),
    ("Je m'appelle Marie", "My name is Marie"),
    ("Comment ça va?", "How are you?"),
    ("J'aime le chocolat", "I love chocolate"),
    ("Il fait beau aujourd'hui", "The weather is nice today"),
    ("Je suis fatigué", "I am tired"),
    ("Où est la bibliothèque?", "Where is the library?"),
    ("Je voudrais un café", "I would like a coffee"),
    ("Parlez-vous anglais?", "Do you speak English?"),
    ("Quelle heure est-il?", "What time is it?"),
    ("J'habite à Paris", "I live in Paris"),
    ("C'est une bonne idée", "That's a good idea"),
    ("Merci beaucoup", "Thank you very much"),
    ("Je ne comprends pas", "I do not understand"),
    ("Pouvez-vous répéter?", "Can you repeat?"),
    ("J'ai besoin d'aide", "I need help"),
    ("Je suis étudiant", "I am a student"),
    ("C'est délicieux", "It's delicious"),
    ("Félicitations", "Congratulations"),
    ("Bonne chance", "Good luck")
]

# Function to generate and save audio using gTTS
def generate_audio(text, filename):
    try:
        # Initialize gTTS for French text
        tts = gTTS(text, lang='fr')
        tts.save(filename)
        print(f"Saved: {filename}")
    except Exception as e:
        print(f"Error generating audio for {text}: {e}")

# Save audio files and metadata
data = []
for i, (fr_text, en_text) in enumerate(phrases, 1):
    filename = f"audio{i}.mp3"
    filepath = os.path.join(dataset_dir, filename)

    # Generate the audio for French text
    generate_audio(fr_text, filepath)

    # Append metadata
    data.append((filename, fr_text, en_text))

# Add a small delay before saving files to ensure audio is saved properly
time.sleep(2)

# Save the metadata as a CSV file
df = pd.DataFrame(data, columns=["Audio File", "French Text", "English Translation"])
csv_filepath = os.path.join(dataset_dir, "references.csv")
df.to_csv(csv_filepath, index=False)

print("\n✅ Dataset generation complete. Check the audio files and references.csv.")


Saved: french_audio_dataset/audio1.mp3
Saved: french_audio_dataset/audio2.mp3
Saved: french_audio_dataset/audio3.mp3
Saved: french_audio_dataset/audio4.mp3
Saved: french_audio_dataset/audio5.mp3
Saved: french_audio_dataset/audio6.mp3
Saved: french_audio_dataset/audio7.mp3
Saved: french_audio_dataset/audio8.mp3
Saved: french_audio_dataset/audio9.mp3
Saved: french_audio_dataset/audio10.mp3
Saved: french_audio_dataset/audio11.mp3
Saved: french_audio_dataset/audio12.mp3
Saved: french_audio_dataset/audio13.mp3
Saved: french_audio_dataset/audio14.mp3
Saved: french_audio_dataset/audio15.mp3
Saved: french_audio_dataset/audio16.mp3
Saved: french_audio_dataset/audio17.mp3
Saved: french_audio_dataset/audio18.mp3
Saved: french_audio_dataset/audio19.mp3
Saved: french_audio_dataset/audio20.mp3

✅ Dataset generation complete. Check the audio files and references.csv.


In [19]:
import os
import pandas as pd
import whisper
from transformers import pipeline
from jiwer import wer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize
from gtts import gTTS
import nltk
import time
from pydub import AudioSegment

# Download required NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')

# Load the Whisper model (medium or large model for better accuracy)
asr_model = whisper.load_model("medium")  # Use "medium" or "large" for better accuracy
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")

# Load reference data
df = pd.read_csv("/content/french_audio_dataset/references.csv")

# Prepare to store results
results = []

for i, row in df.iterrows():
    file = row["Audio File"]
    ref_fr = row["French Text"]
    ref_en = row["English Translation"]

    print(f"Processing: {file}")
    audio_path = os.path.join("french_audio_dataset", file)

    # Convert MP3 to WAV for better transcription quality (if needed)
    if file.endswith(".mp3"):
        sound = AudioSegment.from_mp3(audio_path)
        wav_path = audio_path.replace(".mp3", ".wav")
        sound.export(wav_path, format="wav")
        audio_path = wav_path  # Update to the new .wav file

    # Add a small delay before processing the audio file
    time.sleep(0.5)

    # Step 1: Transcribe
    try:
        hyp_fr = asr_model.transcribe(audio_path, language="fr", task="transcribe")["text"]
    except RuntimeError as e:
        print(f"Error transcribing {file}: {e}")
        hyp_fr = "Transcription Error"

    # Only proceed if transcription was successful
    if hyp_fr != "Transcription Error":
        # Step 2: Translate
        hyp_en = translator(hyp_fr, max_length=500)[0]['translation_text']

        # Step 3: Synthesize (Optional, TTS)
        try:
            tts = gTTS(hyp_en, lang='en')
            tts.save(f"output_{i+1}.mp3")
        except Exception as e:
            print(f"Error synthesizing audio for {file}: {e}")

        # Step 4: Evaluation - Compute WER and BLEU
        wer_score = wer(ref_fr.lower(), hyp_fr.lower())
        ref_tokens = [word_tokenize(ref_en.lower())]
        hyp_tokens = word_tokenize(hyp_en.lower())
        bleu = sentence_bleu(ref_tokens, hyp_tokens, smoothing_function=SmoothingFunction().method4)
    else:
        # If transcription failed, assign placeholder values for evaluation metrics
        hyp_en = "Translation Error"
        wer_score = float('inf')  # Indicate a very high error
        bleu = 0.0  # Indicate no similarity

    # Append the results to the list
    results.append({
        "filename": file,
        "ref_french": ref_fr,
        "hyp_french": hyp_fr,
        "ref_english": ref_en,
        "hyp_english": hyp_en,
        "WER": round(wer_score, 3) if wer_score != float('inf') else 'N/A',
        "BLEU": round(bleu, 3)
    })

# Save the results to a CSV file
results_df = pd.DataFrame(results)
results_df.to_csv("evaluation_results.csv", index=False)

# Print completion message
print("\n✅ Evaluation complete. Check 'evaluation_results.csv'")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


Processing: audio1.mp3


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processing: audio2.mp3


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


...

Processing: audio19.mp3


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processing: audio20.mp3


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



✅ Evaluation complete. Check 'evaluation_results.csv'
